# Install Conda Environments in OpenSARlab using an environment.yml

**If you wish to create a conda environment with the conda create commnand, use this [notebook](../opensarlab_docs/OpenSARlab_supplements/Jupyter_Conda_Environments.ipynb) instead.**

In [ ]:
import glob
import ipywidgets
import os

from IPython.display import Markdown
from IPython.display import display

### List Currently Installed Conda Environments

In [ ]:
%conda env list

### Select a Conda Environment to Create

Note: new environment.yml files should be stored in /home/jovyan/Environment_Configs/ and follow the naming convention \<environment_name\>_env.yml where \<environment_name\> is the environment name

In [ ]:
env_ymls = glob.glob("/home/jovyan/conda_environments/Environment_Configs/*.yml")
height = len(env_ymls) * 19
names =  ipywidgets.RadioButtons(
    options=[ os.path.basename(f).split('_env')[0] for f in env_ymls ],
    description="",
    disabled=False,
    layout=ipywidgets.Layout(height=f"{height}px", width='500px')
)
names

### Collect the Environment Name, Prefix, and Path to its env.yml

In [ ]:
yml = ""
name = names.value
for pth in env_ymls:
    if f"{name}_env.yml" in pth:
        yml = pth
        break
prefix = f"/home/jovyan/.local/envs/{name}"
print(yml)

### Create or the Environment
This will overwrite an existing enviornment but is safer than an update for environments that change PATH and/or PYTHONPATH

In [ ]:
%conda env create -f $yml --prefix $prefix --force

### Update iykernel with the environment name and location of its kernel

In [ ]:
%conda run -n $name kernda --display-name $name -o $prefix/share/jupyter/kernels/python3/kernel.json

### Complete installations and repository clones for newly created insar_analysis and train environments

In [ ]:
if env_cmd.value == 'Create':
    if name == "insar_analysis":
        !source install_insar_analysis_pkgs.sh
    elif name == "train":
        !source train.sh